In [86]:
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/identify_Iris/iris_train.csv", index_col = 'id')
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/identify_Iris/iris_test.csv", index_col = 'id')
train.head()
test.head()

,species,sepal length (cm),petal length (cm)
id,,,
0,setosa,5.4,1.7
1,setosa,5.7,1.5
2,setosa,5.3,1.5
3,setosa,5.1,1.9
4,virginica,6.0,4.8


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [87]:
all_data = pd.concat([train, test], sort=False)

all_data.count()

species              150
sepal length (cm)    150
petal length (cm)    150
sepal width (cm)      75
petal width (cm)      75
dtype: int64

In [88]:
names = all_data['species'].unique()
count = 1

for n in names:
  all_data.loc[all_data['species'] == n, 'species(status)'] = count
  count += 1

all_data.head()

,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),species(status)
id,,,,,,
0,setosa,4.4,1.4,2.9,0.2,1.0
1,versicolor,6.4,4.5,3.2,1.5,2.0
2,virginica,6.2,4.8,2.8,1.8,3.0
3,virginica,7.2,6.1,3.6,2.5,3.0
4,setosa,4.9,1.4,3.0,0.2,1.0


In [89]:

for n in names:
  sum = all_data.loc[(all_data['sepal width (cm)'].notnull()) & (all_data['species'] == n), 'sepal width (cm)'].sum()
  count = len(all_data.loc[(all_data['sepal width (cm)'].notnull()) & (all_data['species'] == n), 'sepal width (cm)'])
  
  all_data.loc[all_data['species'] == n, 'sepal width(ave)'] = sum/count


  sum = all_data.loc[(all_data['petal width (cm)'].notnull()) & (all_data['species'] == n), 'petal width (cm)'].sum()
  count = len(all_data.loc[(all_data['petal width (cm)'].notnull()) & (all_data['species'] == n), 'petal width (cm)'])
  
  all_data.loc[all_data['species'] == n, 'petal width(ave)'] = sum/count

all_data.head()

,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),species(status),sepal width(ave),petal width(ave)
id,,,,,,,,
0,setosa,4.4,1.4,2.9,0.2,1.0,3.346667,0.200000
1,versicolor,6.4,4.5,3.2,1.5,2.0,2.790000,1.353333
2,virginica,6.2,4.8,2.8,1.8,3.0,2.933333,2.006667
3,virginica,7.2,6.1,3.6,2.5,3.0,2.933333,2.006667
4,setosa,4.9,1.4,3.0,0.2,1.0,3.346667,0.200000


In [90]:
all_data['sepal (rate)'] = all_data['sepal width(ave)'] / all_data['sepal length (cm)']
all_data['petal (rate)'] = all_data['petal width(ave)'] / all_data['petal length (cm)']

In [91]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

In [92]:
train.corr()

,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),species(status),sepal width(ave),petal width(ave),sepal (rate),petal (rate)
sepal length (cm),1.000000,0.864852,0.027104,0.763597,0.705445,-0.555266,0.727051,-0.918102,0.451606
petal length (cm),0.864852,1.000000,-0.274590,0.953549,0.920749,-0.728839,0.949857,-0.907358,0.755078
sepal width (cm),0.027104,-0.274590,1.000000,-0.217103,-0.303901,0.529619,-0.377409,0.204403,-0.559515
petal width (cm),0.763597,0.953549,-0.217103,1.000000,0.937646,-0.679398,0.953402,-0.828749,0.796922
species(status),0.705445,0.920749,-0.303901,0.937646,1.000000,-0.573811,0.983474,-0.744990,0.854648
sepal width(ave),-0.555266,-0.728839,0.529619,-0.679398,-0.573811,1.000000,-0.712604,0.825005,-0.772548
petal width(ave),0.727051,0.949857,-0.377409,0.953402,0.983474,-0.712604,1.000000,-0.820556,0.902895
sepal (rate),-0.918102,-0.907358,0.204403,-0.828749,-0.744990,0.825005,-0.820556,1.000000,-0.670939
petal (rate),0.451606,0.755078,-0.559515,0.796922,0.854648,-0.772548,0.902895,-0.670939,1.000000


In [93]:
train.columns

Index(['species', 'sepal length (cm)', 'petal length (cm)', 'sepal width (cm)',
       'petal width (cm)', 'species(status)', 'sepal width(ave)',
       'petal width(ave)', 'sepal (rate)', 'petal (rate)'],
      dtype='object')

In [ ]:

train.loc[train['sepal length (cm)'] < 7.7, 'sepal_len'] = 4
train.loc[train['sepal length (cm)'] < 6.6, 'sepal_len'] = 3
train.loc[train['sepal length (cm)'] < 6.1, 'sepal_len'] = 2
train.loc[train['sepal length (cm)'] < 5.55, 'sepal_len'] = 1

test.loc[test['sepal length (cm)'] < 7.7, 'sepal_len'] = 4
test.loc[test['sepal length (cm)'] < 6.6, 'sepal_len'] = 3
test.loc[test['sepal length (cm)'] < 6.1, 'sepal_len'] = 2
test.loc[test['sepal length (cm)'] < 5.55, 'sepal_len'] = 1

train.head()

,species,sepal length (cm),petal length (cm),sepal width (cm),petal width (cm),sepal_len
id,,,,,,
0,setosa,4.4,1.4,2.9,0.2,1.0
1,versicolor,6.4,4.5,3.2,1.5,3.0
2,virginica,6.2,4.8,2.8,1.8,3.0
3,virginica,7.2,6.1,3.6,2.5,4.0
4,setosa,4.9,1.4,3.0,0.2,1.0


In [94]:
x_col = ['sepal length (cm)', 'petal length (cm)', 'species(status)', 'sepal width(ave)', 'sepal (rate)', 'petal (rate)']
y_col = ['sepal width (cm)', 'petal width (cm)']

x_train = train[x_col]
y_train = train[y_col]

x_test = test[x_col]

In [64]:
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


xgb = XGBRegressor(random_state=42)
gbr = GradientBoostingRegressor(random_state=42)
rfr = RandomForestRegressor(random_state=42)
lr = LinearRegression()


#model_names = ['XGBRegressor', 'GradientBoostingRegressor', 'RandomForestRegressor', 'LinearRegression']
#models = [xgb, gbr, rfr, lr]
model_names = ['RandomForestRegressor', 'LinearRegression']
models = [rfr, lr]


for ms, mn in zip(models, model_names):
  print(mn)
  ms.fit(x_train, y_train)
  pred = ms.predict(x_test)

  print(mn+" : "+(str)(mean_squared_error(y_train, pred, squared=False))+"\r\n")

RandomForestRegressor
RandomForestRegressor : 0.846552313519918

LinearRegression
LinearRegression : 0.8314230230129535



In [95]:
submission = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/identify_Iris/sample_submission.csv", index_col = 'id')


In [96]:
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

In [97]:
submission[y_col] = pred

In [98]:
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/identify_Iris/Iris.csv", index=True)

# New Section